# Assemble green function with ball drop observation
2021.05.09 Kurama Okubo

- 2022.11.23 update for new ball-drop test 
- 2024.1.24 update for master plot
- 2024.7.17 update for the evaluation of attenuation factor (large Qp and Qs)
- 2025.3.16 update for master plot

## Work flow

1. Read isocoord pickle associated with stations
2. Read meta data
3. Read synthetic waveform
4. Preprocess (scaling with M0: scaling factor used in OpenSWPC; convert from disp to vel; time shift + zero padding)
4. Save pickle

In [ ]:
import os
import obspy
from obspy import read, Stream, Trace
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from glob import glob
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm
import warnings

from obspy.core.utcdatetime import UTCDateTime  
os.environ['TZ'] = 'GMT' # change time zone to avoid confusion in unix_tvec conversion
UTCDateTime.DEFAULT_PRECISION = 8

In [ ]:
#Parameters
rootdir = "../../SensorCoupling_BallDrop/code/"

simulationtitle = 'balltest_sidecoord_segment_test' # simulation title used in Input.inf of OpenSWPC
fi_hetzsourceprm = './04_numericalmodeling_waveform/4m_sidecoord_v01_Qp10000_Qs10000_v2_airQ10/out/stf_greenstf_prm.dat'

vmean = 6200

In [ ]:
# Path for event location table

eventloc_table = rootdir+"../data/balldrop_locations.csv"

# pickle event waveform directory
eventdatadir = '../data/DATA_isocoord' # use the one with large pretrigger

# greens function data directory 
# greendatadir = rootdir+'./04_numericalmodeling_waveform/4m_sidecoord_v01_Qp200_Qs80_v2/out/green/OL00'
greendatadir = './04_numericalmodeling_waveform/4m_sidecoord_v01_Qp10000_Qs10000_v2_airQ10/out/green/OL00'

# output datadir
outputdir = "../data/bkcheck_DATA_greencomparison"

# channel table
channel_finame = '../../../Others/AEchanneltable/AEsensorlocation_onFB03_table.csv'


In [ ]:
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

In [ ]:
# read gindex table
df_gindextable = pd.read_csv(rootdir+"../data/gindex_table.csv", index_col=0)
df_gindextable.head()

In [ ]:
# read observation casename  table
df_iso_all = pd.read_csv(rootdir+"../data/balldrop_events_isocoord.csv", index_col=0)
df_iso_all.head()

In [ ]:
# Read Channel Index
df_array = pd.read_csv(channel_finame)

channel_loc={}

for i in range(len(df_array)):
    stnm = df_array.iloc[i].Instrument_Label
    xtemp = df_array.iloc[i].North.astype('float')
    ytemp = df_array.iloc[i].East.astype('float')
    ztemp = df_array.iloc[i].Down.astype('float')
    channel_loc[stnm] = [xtemp, ytemp, ztemp]
    
AEsensors = list(channel_loc.keys())

In [ ]:
df_iso_all[df_iso_all.OL == 'OL02']

In [ ]:
# Read OpenSWPC output for Hertzian source parameter
df_hertzprm = pd.read_csv(fi_hetzsourceprm, skipinitialspace=True)

print(f'T0={df_hertzprm["T0"].values[0]*1e6:.1f}us, Tc{df_hertzprm["tc"].values[0]*1e6:.2f}us, hertz_fmax={df_hertzprm["hertz_fmax"].values[0]:.2f}N, fz={df_hertzprm["fz"].values[0]:.6f}')

In [ ]:
df_hertzprm

In [ ]:
df_hertzprm["T0"]*1e6

## Assemble observation and synthetic waveform

In [ ]:
# Read observation
for stnm in tqdm(AEsensors):
# stnm = AEsensors[8]

    foname = os.path.join(outputdir, "{}_bdwaveform_longpretrig.pickle".format(stnm)) 

    if os.path.exists(foname):
        print("{}_bdwaveform.pickle already exists. skip this case.".format(stnm))
#         continue

    df_iso_st = df_iso_all[df_iso_all.OL==stnm]
    xlimit = [-0.5, 3]
    ylimit = [-1.0, 1.0]

    st_eventsta = Stream()

    #---Start assembling each balldrop event---#
    for dataind in df_iso_st.index:
        #---Read observation data---#
        tr_obs_trim = read(eventdatadir+"/obs_isocoord_{}.pickle".format(dataind))[0]
        tr_obs_trim.stats.dataindex = dataind
        # rename channel
        tr_obs_trim.stats.network = 'BIAX' # 'OY': Observatoin in Y direction
        tr_obs_trim.stats.channel = 'OY' # 'OY': Observatoin in Y direction
        st_eventsta.append(tr_obs_trim)


    for tr_obstmp in st_eventsta:
        #---Read synthetic data---#
        dataind = tr_obstmp.stats.dataindex

        OL, datacase = dataind.split('__')

        # read synthetic green's function
        gid = df_gindextable[df_gindextable['index'] == dataind]['gid'].values[0]

        finame_syn = '{}/{}__{:08d}__OL00__z__fy___.sac'.format(greendatadir, simulationtitle, gid) #y coordinate is corresponding to the perpendicular to the fault surface
        tr_syn = read(finame_syn, format="SAC")[0]

        # restore stats
        tr_syn.stats.network = 'OpenSWPC'
        tr_syn.stats.station = OL
        tr_syn.stats.location = tr_obstmp.stats.location
        tr_syn.stats.channel = 'SY' # NOTE: The original name 'G_Vz_fy_' is misleading; The output of green function in OpenSWPC is not in velocity, but displacement.
        tr_syn.stats.dataindex = dataind
        tr_syn.stats.fz_hetzscale = df_hertzprm['fz'].values[0]

        # rescale source scaling and convert from nm to m
        tr_syn.data = tr_syn.data*tr_syn.stats.fz_hetzscale/1e9

        # store the displacement data for the gain comparison
        tr_syn.data_disp = tr_syn.data
        # convert from displacement to velocity
        tr_syn.differentiate() # NOTE: obspy differentiate uses delta in stats for differentiation.

        # manipulate start and endtime
        # 1. zero pad pretriger and windowlen
        # 2. change starttime

        T0 = df_hertzprm['T0'].values[0] # Origin time used in OpenSWPC

        st_tmp = tr_syn.stats.starttime
        pt = timedelta(milliseconds = tr_obstmp.stats.pretrigger)
        wt = timedelta(milliseconds = tr_obstmp.stats.windowlen)
        ot = timedelta(milliseconds = T0)

        starttime_tmp = st_tmp+ot-pt
        endtime_tmp = st_tmp+ot+wt

        tr_syn_trim = tr_syn.copy() # avoid error in multiple trim
        tr_syn_trim.trim(starttime_tmp, endtime_tmp, pad=True, fill_value=0.0)
        
        tr_syn_trim.stats.starttime =  tr_obstmp.stats.starttime
        
        st_eventsta.append(tr_syn_trim)

    # Save Stream containing a set of balldrop observatoin and synthetic waveform
    if not os.path.exists(outputdir):
        os.makedirs(outputdir) 
        
    st_eventsta.write(foname, format="pickle")

In [ ]:
# check comparison

finame = os.path.join(outputdir, "{}_bdwaveform_longpretrig.pickle".format(AEsensors[12])) 
st = read(finame)
st

In [ ]:
tr_obs = st.select(network="BIAX", location='3100')[0]
tr_syn = st.select(network="OpenSWPC", location='3100')[0] 

In [ ]:
tr_obs.stats

In [ ]:
tvec = tr_syn.times() * 1e3

xlimit = [0, 0.2] #[-1.5, 20.5]
ylimit = [-1.2, 1.2]
fig, ax = plt.subplots(1, 1, figsize=(8, 3))

ax.plot(tvec, tr_obs.data*1e3, 'k-', label="observation")
ax.plot(tvec, tr_syn.data*1e3, 'r-', label="synthetic")
ax.axvline(tr_obs_trim.stats.tpick, c = 'b', ls = '--')
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Velocity [mm/s]")
ax.set_xlim(xlimit)
ax.set_ylim(ylimit)
ax.legend()

In [ ]:
tr_syn_trim.plot()